STEP 1: Pre-Processing Starter Notebook


In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

# Define directories
train_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train'
test_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test/BTTAIxNYBG-test'
validation_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation'

# Load datasets
train_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test.csv')
validate_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation.csv')

# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

# Note: No augmentation for validation and test data, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Convert dataframe to a format suitable for the model training
def df_to_dataset(dataframe, datagen, directory, batch_size=32):
    return datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='imageFile',  # Column in dataframe that contains the filenames
        y_col='classLabel',  # Column in dataframe that contains the class/label
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'  # Change this if not a multiclass classification
    )

# Create datasets for training, validation, and testing
train_dataset = df_to_dataset(train_df, train_datagen, train_dir)
validation_dataset = df_to_dataset(validate_df, validation_datagen, validation_dir)

# This setup is now ready for training with model.fit using the train_dataset and validation_dataset

2024-03-17 03:36:46.640062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 03:36:46.640202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 03:36:46.730236: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 81946 validated image filenames belonging to 10 classes.
Found 10244 validated image filenames belonging to 10 classes.


Implement incremental learning 

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2

# Load the MobileNetV2 model, pretrained on ImageNet, without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Create the model
model = Sequential([
    base_model,
    # Convert features to vectors
    tf.keras.layers.GlobalAveragePooling2D(),
    # Add a dense layer for classification
    Dense(1024, activation='relu'),
    # Final layer with softmax activation for multi-class classification
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 [==============================] - 1s 0us/step


In [3]:
# Start with a small portion (e.g., 10%) of the training data
train_subset = train_df.sample(frac=0.1, random_state=42)
train_subset_dataset = df_to_dataset(train_subset, train_datagen, train_dir)


Found 8195 validated image filenames belonging to 10 classes.


In [4]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_subset_dataset,  # Use the subset of data
    validation_data=validation_dataset,
    epochs=1,  # Initially train for fewer epochs for debugging
    callbacks=callbacks
)


I0000 00:00:1710647141.684481      99 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


257/257 [==============================] - ETA: 0s - loss: 0.5264 - accuracy: 0.8266

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


257/257 [==============================] - 376s 1s/step - loss: 0.5264 - accuracy: 0.8266 - val_loss: 0.5216 - val_accuracy: 0.8290


In [5]:
# Start with a small portion (e.g., 10%) of the training data
train_subset = train_df.sample(frac=0.2, random_state=42)
train_subset_dataset = df_to_dataset(train_subset, train_datagen, train_dir)


Found 16389 validated image filenames belonging to 10 classes.


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_subset_dataset,  # Use the subset of data
    validation_data=validation_dataset,
    epochs=1,  # Initially train for fewer epochs for debugging
    callbacks=callbacks
)


513/513 [==============================] - ETA: 0s - loss: 0.3185 - accuracy: 0.8880

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


513/513 [==============================] - 428s 835ms/step - loss: 0.3185 - accuracy: 0.8880 - val_loss: 0.4133 - val_accuracy: 0.8534


STEP 5: Evaluate the Model

In [7]:
validation_loss, validation_accuracy = model.evaluate(validation_dataset)
print(f'Validation Loss: {validation_loss}')
print(f'Validation Accuracy: {validation_accuracy}')

321/321 [==============================] - 84s 261ms/step - loss: 0.4133 - accuracy: 0.8534
Validation Loss: 0.41327783465385437
Validation Accuracy: 0.8533775806427002


STEP 6: Make Predictions on Test Set

In [8]:
test_dataset = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='imageFile',  # Make sure this column name matches your test_df column name for filenames
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,  # No labels
    shuffle=False
)

Found 30690 validated image filenames.


In [9]:
predictions = model.predict(test_dataset)
predicted_class_indices = np.argmax(predictions, axis=1)

960/960 [==============================] - 472s 491ms/step


STEP 7: Prepare Submission File

In [14]:
labels = (train_subset_dataset.class_indices)
labels = dict((v,k) for k,v in labels.items())
predicted_class_ids = [labels[v] for v in predicted_class_indices]

submission_df = pd.DataFrame({'uniqueID': test_df['uniqueID'], 'classID': predicted_class_indices})
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

In [12]:
submission_df

,uniqueID,classID
0,1,1
1,9,7
2,10,4
3,14,1
4,16,6
...,...,...
30685,122864,7
30686,122868,1
30687,122871,0
30688,122878,1


In [15]:
!ls /kaggle/working

best_model.h5  submission.csv
